In [1]:
import matplotlib.pyplot as plt
from matplotlib import pyplot
from configparser import ConfigParser
import pandas as pd
import numpy as np
import pickle
import os
import re
import gc
import csv
from pathlib import Path
import seaborn as sns
#from rfpimp import permutation_importances
import warnings
from tqdm import tqdm
from time import gmtime, strftime
from io import StringIO
from sklearn.linear_model import BayesianRidge
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
#import dill
warnings.filterwarnings('ignore')
#shap.initjs()
pd.options.display.max_columns=1000
pd.options.display.max_rows=1000


In [2]:
#Define missing values.
def missing_values_with_ther(df,perc):
    #Total missing values
    mis_val=df.isnull().sum()
    mis_val=pd.DataFrame({'Column_Name':mis_val.index,'Missing_value':mis_val.values})
    #Percentage missing value
    mis_val_percent=100*df.isnull().sum()/len(df)
    mis_val_percent=pd.DataFrame({'Column_Name':mis_val_percent.index,'Perc_of_Total_values':mis_val_percent.values})
    #table with results
    mis_val_table=pd.concat([mis_val,mis_val_percent],axis=1,ignore_index=True)
    #rename and select required columns
    mis_val_table_with_column=mis_val_table.rename(columns=
                                                  {0:'Column_Name',1:'Missing_value',3:'Perc_of_Total_values'})
    mis_val_table_with_column=mis_val_table_with_column[['Column_Name','Missing_value','Perc_of_Total_values']]
    
    #Sort tabels by %of columns.
    mis_val_table_with_column=mis_val_table_with_column[mis_val_table_with_column.iloc[:,1]!=0].sort_values(
        'Perc_of_Total_values',ascending=False).round(1)
    
    mis_val_table_with_column=mis_val_table_with_column[mis_val_table_with_column['Perc_of_Total_values']>perc]
    
    #print summary infromation
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_with_column.shape[0]) +
              " columns that have missing values greater than " + str(perc) + " Percentage")
    return mis_val_table_with_column

#Define missing values.
def missing_values(df):
    #Total missing values
    mis_val=df.isnull().sum()
    mis_val=pd.DataFrame({'Column_Name':mis_val.index,'Missing_value':mis_val.values})
    #Percentage missing value
    mis_val_percent=100*df.isnull().sum()/len(df)
    mis_val_percent=pd.DataFrame({'Column_Name':mis_val_percent.index,'Perc_of_Total_values':mis_val_percent.values})
    #table with results
    mis_val_table=pd.concat([mis_val,mis_val_percent],axis=1,ignore_index=True)
    #rename and select required columns
    mis_val_table_with_column=mis_val_table.rename(columns=
                                                  {0:'Column_Name',1:'Missing_value',3:'Perc_of_Total_values'})
    mis_val_table_with_column=mis_val_table_with_column[['Column_Name','Missing_value','Perc_of_Total_values']]
    
    #Sort tabels by %of columns.
    mis_val_table_with_column=mis_val_table_with_column[mis_val_table_with_column.iloc[:,1]!=0].sort_values(
        'Perc_of_Total_values',ascending=False).round(1)
    
    #mis_val_table_with_column=mis_val_table_with_column[mis_val_table_with_column['Perc_of_Total_values']>perc]
    
    #print summary infromation
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_with_column.shape[0]) +
              " columns that have missing values.")
    return mis_val_table_with_column


# Removal of Zero Variance Columns
def rem_var(df,ther):
    numeric_df=df.select_dtypes(exclude='object')
    numeric_df=numeric_df[numeric_df.columns.drop(list(['churn','Customer_ID']))]
    
    #Remove near zero variance columns
    selector=VarianceThreshold(ther)
    selector.fit(numeric_df)
    
    numeric_df_1=df[numeric_df.columns[selector.get_support(indices=True)]]
    
    print ("Your selected dataframe has " + str(numeric_df.shape[1]) + " columns.\n"      
           "Your selected dataframe has " + str(numeric_df.shape[1]) + " Numerical columns.\n" 
            "There are " + str(numeric_df_1.shape[1]) +" columns has variance greater than " + str(ther) + " Percentage")
      
    object_df = df.select_dtypes(include='object')
    
    df_1 = pd.concat([df[['Customer_ID','churn']],df[object_df.columns],numeric_df_1], axis=1)
    
    return df_1

#Removing the correlated Columns.
def correlated(df,val):
    
    #Only Numerical columns
    numeric_df=df.select_dtypes(exclude='object')
    numeric_df=numeric_df[numeric_df.columns.drop(list(['churn','Customer_ID']))]
    
    # Create correlation matrix
    corr_matrix = numeric_df.corr()
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > val)]
    numeric_df_1 = numeric_df.drop(to_drop, axis=1)
    
    #Categorical Columns
    object_df = df.select_dtypes(include='object')
    df_1 = pd.concat([df[['Customer_ID','churn']],df[object_df.columns],numeric_df_1], axis=1)

    gc.enable()
    del corr_matrix
    gc.collect()
    
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
           "In that dataframe has " + str(numeric_df.shape[1]) + " Numerical columns.\n"
           "There are " + str(len(to_drop)) +" columns has correlation greater than " + str(val) + " Percentage")
    
    print(numeric_df_1.shape)
    return df_1


def data_outlier(df,low_cut,high_cut):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numeric_col=df[df.columns.drop(list(['churn','Customer_ID']))].select_dtypes(include=numerics).columns
    i=0
    for i in range(len(numeric_col)):
        column=numeric_col[i]
        df.loc[df[column]>df[column].quantile(high_cut),column]=df[column].quantile(high_cut)
        df.loc[df[column]<df[column].quantile(low_cut),column]=df[column].quantile(low_cut)
        
def dummy(df):
    #converting object to category
    col=df[df.columns.difference(['Customer_ID'])].select_dtypes('object').columns
    #col=col.append(pd.Index(['fis_quarter']))
    for col in col:
        df[col]=df[col].astype('category')

    #Extracting categorical columns.
    col=df[df.columns.difference(['Customer_ID'])].select_dtypes('category').columns
    #Creating dummy varaibles
    df=pd.get_dummies(df,columns=col)

    #Renaming the column name with jason characters
    df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))
    
    return df

#Aggregation on the Importance columns that come in the model Importance.

def aggregation_id(df,cat_column):
          
    """
    Groups and aggregates the numeric values in a child dataframe
    by the parent variable.
    
    Parameters
    --------
        churn_df (dataframe): 
            The dataframe to calculate the statistics on
        parent_var (string): 
            the parent variable used for grouping and aggregating
        columns (string): 
            The column on whicg the aggregation is happening
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated by the `parent_var` for 
            all numeric columns. Each observation of the parent variable will have 
            one row in the dataframe with the parent variable as the index. 
            The columns are also renamed using the `df_name`. Columns with all duplicate
            values are removed. 
    
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numeric_col=df[df.columns.difference(['Customer_ID','churn'])].select_dtypes(include=numerics).columns
    numeric_col=numeric_col.append(pd.Index([cat_column]))
    numeric_df = df[numeric_col]
    agg = numeric_df.groupby([cat_column]).agg(['count', 'mean', 'max', 'min', 'sum'])
    # Need to create new column names
    columns = []
    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        if var != cat_column:
            # Iterate through the stat names
            for stat in agg.columns.levels[1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (cat_column,var,stat))

    agg.columns = columns
    
    # Remove the columns with all redundant values
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:, idx]
    
    return agg

In [254]:
#os.getcwd()
#Setting data directory
data_dir = Path('..', 'iim_apds_churn_prjct')

data_path=Path(data_dir,'data/satish/Telecom_customer_churn.csv')
telo_df=pd.read_csv(data_path)
print(telo_df.shape)
telo_df.head()

(100000, 100)


,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,opk_dat_Mean,mou_opkv_Mean,mou_opkd_Mean,drop_blk_Mean,attempt_Mean,complete_Mean,callfwdv_Mean,callwait_Mean,churn,months,uniqsubs,actvsubs,new_cell,crclscod,asl_flag,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,prizm_social_one,area,dualband,refurb_new,hnd_price,phones,models,hnd_webcap,truck,rv,ownrent,lor,dwlltype,marital,adults,infobase,income,numbcars,HHstatin,dwllsize,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID
0,23.9975,219.25,22.500,0.2475,0.00,0.0,0.0,0.0,0.0,-157.25,-18.9975,0.666667,0.0,0.666667,0.0,6.333333,0.0,52.333333,0.0,42.333333,0.0,45.000000,0.0,0.000000,0.000000,0.000000,18.000000,0.000000,90.643333,0.0,97.176667,0.000000,0.000000,0.000000,0.000000,58.000000,0.0,132.600000,0.0,24.000000,0.0,55.220000,0.0,1.333333,52.333333,45.000000,0.0,0.333333,1,61,2,1,U,A,N,1652,4228.00000,1504.62,1453.44,4085.00,1602,29.66,83.37,32.69,272,116,30,322.0,136.0,38.0,S,NORTHWEST/ROCKY MOUNTAIN AREA,Y,N,149.98999,2.0,2.0,WCMB,0.0,0.0,O,15.0,S,S,1.0,M,4.0,3.0,C,A,0.0,N,U,U,U,U,U,Y,361.0,1000001
1,57.4925,482.75,37.425,0.2475,22.75,9.1,9.1,0.0,0.0,532.25,50.9875,8.333333,0.0,1.000000,0.0,61.333333,0.0,263.333333,0.0,69.000000,0.0,193.333333,0.0,1.666667,6.333333,5.463333,53.000000,0.333333,189.396667,0.0,55.280000,46.333333,24.216667,6.333333,3.696667,83.666667,0.0,75.333333,0.0,157.000000,0.0,169.343333,0.0,9.333333,263.333333,193.333333,0.0,5.666667,0,56,1,1,N,EA,N,14654,26400.00000,2851.68,2833.88,26367.00,14624,51.53,479.40,265.89,305,158,40,477.0,275.0,48.0,U,CHICAGO AREA,N,N,NaN,7.0,6.0,WC,1.0,1.0,NaN,1.0,S,S,1.0,M,5.0,1.0,C,A,0.0,Z,U,U,U,U,U,Y,240.0,1000002
2,16.9900,10.25,16.990,0.0000,0.00,0.0,0.0,0.0,0.0,-4.25,0.0000,0.333333,0.0,0.000000,0.0,2.666667,0.0,9.000000,0.0,0.333333,0.0,6.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,5.426667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.0,5.193333,0.0,1.000000,0.0,0.233333,0.0,0.333333,9.000000,6.000000,0.0,0.000000,1,58,1,1,Y,C,N,7903,24385.05333,2155.91,1934.47,24303.05,7888,34.54,433.98,140.86,12,7,17,11.0,6.0,17.0,S,GREAT LAKES AREA,N,N,29.98999,2.0,1.0,NaN,0.0,0.0,O,7.0,S,M,2.0,M,5.0,2.0,C,A,0.0,N,U,Y,U,U,U,Y,1504.0,1000003
3,38.0000,7.50,38.000,0.0000,0.00,0.0,0.0,0.0,0.0,-1.50,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.0,3.666667,0.0,1.333333,0.0,3.666667,0.0,0.000000,0.000000,0.000000,1.333333,0.000000,8.410000,0.0,0.413333,0.333333,0.256667,0.000000,0.000000,1.333333,0.0,3.380000,0.0,3.666667,0.0,5.450000,0.0,0.000000,3.666667,3.666667,0.0,0.000000,0,60,1,1,Y,B,N,1502,3065.00000,2000.90,1941.81,3035.00,1479,40.45,63.23,30.81,8,3,38,50.0,25.0,40.0,T,CHICAGO AREA,N,N,29.98999,1.0,1.0,NaN,0.0,0.0,NaN,6.0,M,M,4.0,M,6.0,1.0,C,D,0.0,U,Y,U,U,U,U,Y,1812.0,1000004
4,55.2300,570.50,71.980,0.0000,0.00,0.0,0.0,0.0,0.0,38.50,0.0000,9.666667,0.0,0.666667,0.0,77.000000,0.0,222.333333,0.0,94.666667,0.0,137.000000,0.0,8.666667,15.000000,11.076667,66.000000,0.000000,285.233333,0.0,106.330000,14.666667,10.816667,0.666667,0.366667,97.333333,0.0,173.476667,0.0,90.333333,0.0,218.086667,0.0,10.333333,222.333333,137.000000,0.0,0.000000,0,57,1,1,Y,A,N,4485,14028.00000,2181.12,2166.48,13965.00,4452,38.69,249.38,79.50,558,191,55,586.0,196.0,80.0,U,NEW ENGLAND AREA,Y,N,149.98999,6.0,4.0,WCMB,0.0,0.0,R,5.0,M,S,1.0,M,6.0,1.0,C,O,0.0,I,U,U,U,U,U,Y,434.0,1000005


In [243]:
missing_values_with_ther(telo_df,10)

Your selected dataframe has 100 columns.
There are 10 columns that have missing values greater than 10 Percentage


,Column_Name,Missing_value,Perc_of_Total_values
87,numbcars,49366,49.4
89,dwllsize,38308,38.3
88,HHstatin,37923,37.9
80,ownrent,33706,33.7
82,dwlltype,31909,31.9
81,lor,30190,30.2
86,income,25436,25.4
84,adults,23019,23.0
85,infobase,22079,22.1
77,hnd_webcap,10189,10.2


In [255]:
#Imputing mean for the missing numerical variables.
telo_df_missing=missing_values(telo_df)
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_col=telo_df[telo_df_missing['Column_Name']].select_dtypes(include=numerics).columns

#Imputing missing value by imputer from sklearn.
imp = IterativeImputer(random_state=0,missing_values=np.nan)
imp.fit(telo_df[numeric_col])
telo_df[numeric_col]=imp.transform(telo_df[numeric_col])

telo_df.shape

Your selected dataframe has 100 columns.
There are 43 columns that have missing values.


(100000, 100)

In [231]:
#Imputing mean for the missing numerical variables.
telo_df_missing=missing_values(telo_df)
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_col=telo_df[telo_df_missing['Column_Name']].select_dtypes(include=numerics).columns
telo_df[numeric_col]=telo_df[numeric_col].fillna(telo_df[numeric_col].mean())
print(telo_df.shape)

Your selected dataframe has 100 columns.
There are 43 columns that have missing values.
(100000, 100)


In [256]:
#Feature Enginering:

#mou_Mean/rev_Mean --> mean_per_minute_charge
telo_df['fe_mean_per_minute_charge']=telo_df['mou_Mean']/telo_df['rev_Mean']

#rev_Mean-totmrc_Mean-- > mean_extra_amount_they_pay
telo_df['fe_mean_extra_amount_they_pay']=telo_df['rev_Mean']-telo_df['totmrc_Mean']

#mou_Mean-(ovrmou_Mean) -->mean_allocated_calls
telo_df['fe_mean_allocated_calls']=telo_df['mou_Mean']-telo_df['ovrmou_Mean']

#(ovrrev_Mean+vceovr_Mean+datovr_Mean)---> mean_profit
telo_df['fe_mean_profit']=telo_df['ovrrev_Mean']+telo_df['vceovr_Mean']+telo_df['datovr_Mean']

#(drop_vce_Mean+drop_dat_Mean+blck_vce_Mean+blck_dat_Mean) --->tot_mean_failed_calls
telo_df['fe_tot_mean_failed_calls']=telo_df['drop_vce_Mean']+telo_df['drop_dat_Mean']+telo_df['blck_vce_Mean']+telo_df['blck_dat_Mean']
#telo_df=telo_df[telo_df.columns.drop(['blck_dat_Mean'])]

#comp_vce_Mean/plcd_vce_Mean ---> perc_of_success_calls
telo_df['fe_perc_of_success_calls']=telo_df['comp_vce_Mean']/telo_df['plcd_vce_Mean']

#(mou_cvce_Mean-comp_vce_Mean) --> mou_cvce_diff_vce_mean ---> unrounded to rounded completed voice call difference
telo_df['fe_mou_cvce_diff_vce_mean']=telo_df['mou_cvce_Mean']-telo_df['comp_vce_Mean']

#(mou_rvce_Mean-recv_vce_Mean) --> mou_rvce_diff_vce_mean ---> unrounded to rounded completed voice call difference
telo_df['fe_mou_rvce_diff_vce_mean']=telo_df['mou_rvce_Mean']-telo_df['recv_vce_Mean']

#totrev/totcalls ----> tot_revenue_per_call -----> Total Revenue per call
telo_df['fe_tot_revenue_per_call']=telo_df['totrev']/telo_df['totcalls']

# totmou/totcalls ---> tot_mou_per_call ----> Total minutes per call
telo_df['fe_tot_mou_per_call']=telo_df['totmou']/telo_df['totcalls']

# tot_revenue_per_call*tot_mou_per_call ---> tot_charge_per_call  --> what is the total charge per call
telo_df['fe_tot_charge_per_call']=telo_df['fe_tot_revenue_per_call']*telo_df['fe_tot_mou_per_call']

#totrev-adjrev ---> tot_revenue_adj  --> Total revenue adjustment
telo_df['fe_tot_revenue_adj']=telo_df['totrev']-telo_df['adjrev']

#totmou-adjmou  ---> tot_mou_adj  ---> Total minute adjustment
telo_df['fe_tot_mou_adj']=telo_df['totmou']-telo_df['adjmou']

#totcalls-adjqty ---> tot_calls_adj  --> Total call adjustment
telo_df['fe_tot_calls_adj']=telo_df['totcalls']-telo_df['adjqty']

#avgrev/avgqty ----> avg_revenue_per_call -----> Average Revenue per call
telo_df['fe_avg_revenue_per_call']=telo_df['avgrev']/telo_df['avgqty']

# avgmou/avgqty ---> avg_mou_per_call ----> Average minutes per call
telo_df['fe_avg_mou_per_call']=telo_df['avgmou']/telo_df['avgqty']

# avg_revenue_per_call*avg_mou_per_call ---> avg_charge_per_call  --> what is the average charge per call
telo_df['fe_avg_charge_per_call']=telo_df['fe_avg_revenue_per_call']*telo_df['fe_avg_mou_per_call']

telo_df.head()

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,opk_dat_Mean,mou_opkv_Mean,mou_opkd_Mean,drop_blk_Mean,attempt_Mean,complete_Mean,callfwdv_Mean,callwait_Mean,churn,months,uniqsubs,actvsubs,new_cell,crclscod,asl_flag,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,prizm_social_one,area,dualband,refurb_new,hnd_price,phones,models,hnd_webcap,truck,rv,ownrent,lor,dwlltype,marital,adults,infobase,income,numbcars,HHstatin,dwllsize,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID,fe_mean_per_minute_charge,fe_mean_extra_amount_they_pay,fe_mean_allocated_calls,fe_mean_profit,fe_tot_mean_failed_calls,fe_perc_of_success_calls,fe_mou_cvce_diff_vce_mean,fe_mou_rvce_diff_vce_mean,fe_tot_revenue_per_call,fe_tot_mou_per_call,fe_tot_charge_per_call,fe_tot_revenue_adj,fe_tot_mou_adj,fe_tot_calls_adj,fe_avg_revenue_per_call,fe_avg_mou_per_call,fe_avg_charge_per_call
0,23.9975,219.25,22.500,0.2475,0.00,0.0,0.0,0.0,0.0,-157.25,-18.9975,0.666667,0.0,0.666667,0.0,6.333333,0.0,52.333333,0.0,42.333333,0.0,45.000000,0.0,0.000000,0.000000,0.000000,18.000000,0.000000,90.643333,0.0,97.176667,0.000000,0.000000,0.000000,0.000000,58.000000,0.0,132.600000,0.0,24.000000,0.0,55.220000,0.0,1.333333,52.333333,45.000000,0.0,0.333333,1,61,2,1,U,A,N,1652,4228.00000,1504.62,1453.44,4085.00,1602,29.66,83.37,32.69,272,116,30,322.0,136.0,38.0,S,NORTHWEST/ROCKY MOUNTAIN AREA,Y,N,149.989990,2.0,2.0,WCMB,0.0,0.0,O,15.0,S,S,1.0,M,4.0,3.0,C,A,0.0,N,U,U,U,U,U,Y,361.0,1000001,9.136368,1.4975,219.25,0.0,1.333333,0.859873,45.643333,54.843333,0.910787,2.559322,2.330997,51.18,143.00000,50,0.907311,2.550321,2.313935
1,57.4925,482.75,37.425,0.2475,22.75,9.1,9.1,0.0,0.0,532.25,50.9875,8.333333,0.0,1.000000,0.0,61.333333,0.0,263.333333,0.0,69.000000,0.0,193.333333,0.0,1.666667,6.333333,5.463333,53.000000,0.333333,189.396667,0.0,55.280000,46.333333,24.216667,6.333333,3.696667,83.666667,0.0,75.333333,0.0,157.000000,0.0,169.343333,0.0,9.333333,263.333333,193.333333,0.0,5.666667,0,56,1,1,N,EA,N,14654,26400.00000,2851.68,2833.88,26367.00,14624,51.53,479.40,265.89,305,158,40,477.0,275.0,48.0,U,CHICAGO AREA,N,N,109.757449,7.0,6.0,WC,1.0,1.0,NaN,1.0,S,S,1.0,M,5.0,1.0,C,A,0.0,Z,U,U,U,U,U,Y,240.0,1000002,8.396747,20.0675,460.00,18.2,9.333333,0.734177,-3.936667,-13.720000,0.194601,1.801556,0.350584,17.80,33.00000,30,0.193802,1.803001,0.349425
2,16.9900,10.25,16.990,0.0000,0.00,0.0,0.0,0.0,0.0,-4.25,0.0000,0.333333,0.0,0.000000,0.0,2.666667,0.0,9.000000,0.0,0.333333,0.0,6.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,5.426667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.0,5.193333,0.0,1.000000,0.0,0.233333,0.0,0.333333,9.000000,6.000000,0.0,0.000000,1,58,1,1,Y,C,N,7903,24385.05333,2155.91,1934.47,24303.05,7888,34.54,433.98,140.86,12,7,17,11.0,6.0,17.0,S,GREAT LAKES AREA,N,N,29.989990,2.0,1.0,NaN,0.0,0.0,O,7.0,S,M,2.0,M,5.0,2.0,C,A,0.0,N,U,Y,U,U,U,Y,1504.0,1000003,0.603296,0.0000,10.25,0.0,0.333333,0.666667,-0.573333,-0.333333,0.272796,3.085544,0.841725,221.44,82.00333,15,0.245208,3.080931,0.755469
3,38.0000,7.50,38.000,0.0000,0.00,0.0,0.0,0.0,0.0,-1.50,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.0,3.666667,0.0,1.333333,0.0,3.666667,0.0,0.000000,0.000000,0.000000,1.333333,0.000000,8.410000,0.0,0.413333,0.333333,0.256667,0.000000,0.000000,1.333333,0.0,3.380000,0.0,3.666667,0.0,5.450000,0.0,0.000000,3.666667,3.666667,0.0,0.000000,0,60,1,1,Y,B,N,1502,3065.00000,2000.90,1941.81,3035.00,1479,40.45,63.23,30

In [257]:
#Imputing mean for the missing numerical variables.
telo_df_missing=missing_values(telo_df)
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_col=telo_df[telo_df_missing['Column_Name']].select_dtypes(include=numerics).columns
telo_df[numeric_col]=telo_df[numeric_col].fillna(telo_df[numeric_col].mean())
print(telo_df.shape)

Your selected dataframe has 117 columns.
There are 24 columns that have missing values.
(100000, 117)


In [258]:
#Filling as Missing for categorical variables.
telo_df.fillna('Missing',inplace=True)
missing_values(telo_df)
print(telo_df.shape)
gc.collect()

Your selected dataframe has 117 columns.
There are 0 columns that have missing values.
(100000, 117)


40

In [ ]:
# Remaining Missing Categorical variable
telo_df_cat_missing=missing_values(telo_df)
categorical_col=telo_df[telo_df_cat_missing['Column_Name']].select_dtypes(include='object').columns

#Imputing the highest frequency value as for missing values in categorical data.
imp = SimpleImputer(strategy="most_frequent")
telo_df.loc[:,categorical_col]=imp.fit_transform(telo_df.loc[:,categorical_col])

telo_df.head()

In [224]:
#Feature enginiring on categorical variable
#area=aggregation_id(telo_df,'area')
#prizm_social_one=aggregation_id(telo_df,'prizm_social_one')
#ethnic=aggregation_id(telo_df,'ethnic')
#marital=aggregation_id(telo_df,'marital')

#telo_df=telo_df.merge(area,how='left',on=['area'])
#telo_df=telo_df.merge(prizm_social_one,how='left',on=['prizm_social_one'])
#telo_df=telo_df.merge(ethnic,how='left',on=['ethnic'])
#telo_df=telo_df.merge(marital,how='left',on=['marital'])

#print(telo_df.shape)
#telo_df.head()

(100000, 387)


,Customer_ID,churn,new_cell,crclscod,asl_flag,prizm_social_one,area,dualband,refurb_new,hnd_webcap,ownrent,dwlltype,marital,infobase,HHstatin,dwllsize,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,blck_vce_Mean,unan_vce_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,mou_peav_Mean,opk_vce_Mean,mou_opkv_Mean,drop_blk_Mean,attempt_Mean,complete_Mean,callwait_Mean,months,uniqsubs,actvsubs,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,lor,adults,income,numbcars,eqpdays,fe_mean_per_minute_charge,fe_mean_extra_amount_they_pay,fe_mean_allocated_calls,fe_mean_profit,fe_tot_mean_failed_calls,fe_mou_cvce_diff_vce_mean,fe_mou_rvce_diff_vce_mean,fe_tot_revenue_per_call,fe_tot_mou_per_call,fe_tot_charge_per_call,fe_tot_revenue_adj,fe_tot_mou_adj,fe_tot_calls_adj,fe_avg_revenue_per_call,fe_avg_mou_per_call,fe_avg_charge_per_call,marital_change_mou_sum,marital_change_rev_sum,marital_change_mou_min,marital_change_rev_min,marital_change_mou_mean,marital_fe_mean_extra_amount_they_pay_min,marital_change_rev_mean,marital_fe_mou_rvce_diff_vce_mean_min,marital_fe_mou_cvce_diff_vce_mean_min,marital_blck_vce_Mean_min,marital_truck_min,marital_comp_dat_Mean_mean,marital_plcd_dat_Mean_mean,marital_threeway_Mean_mean,marital_fe_avg_revenue_per_call_min,marital_fe_tot_revenue_per_call_min,marital_comp_dat_Mean_max,marital_mou_opkv_Mean_min,marital_truck_mean,marital_roam_Mean_mean,marital_fe_avg_charge_per_call_min,marital_fe_tot_charge_per_call_min,marital_fe_avg_revenue_per_call_mean,marital_fe_tot_revenue_per_call_mean,marital_da_Mean_mean,marital_opk_vce_Mean_min,marital_custcare_Mean_mean,marital_callwait_Mean_mean,marital_truck_max,marital_actvsubs_min,marital_numbcars_min,marital_lor_min,marital_fe_avg_revenue_per_call_max,marital_fe_tot_revenue_per_call_max,marital_actvsubs_mean,marital_models_mean,marital_mou_peav_Mean_min,marital_numbcars_mean,marital_fe_avg_charge_per_call_mean,marital_fe_mean_per_minute_charge_min,marital_uniqsubs_mean,marital_phones_mean,marital_peak_vce_Mean_min,marital_fe_tot_charge_per_call_mean,marital_cc_mou_Mean_mean,marital_fe_tot_mou_per_call_min,marital_fe_avg_mou_per_call_min,marital_adults_min,marital_numbcars_max,marital_actvsubs_max,marital_roam_Mean_max,marital_comp_vce_Mean_min,marital_ccrndmou_Mean_mean,marital_da_Mean_max,marital_mou_cvce_Mean_min,marital_complete_Mean_min,marital_blck_vce_Mean_mean,marital_fe_tot_mou_per_call_mean,marital_fe_avg_mou_per_call_mean,marital_adults_mean,marital_models_max,marital_income_min,marital_attempt_Mean_min,marital_fe_avg_charge_per_call_max,marital_adults_max,marital_drop_vce_Mean_mean,marital_fe_tot_charge_per_call_max,marital_fe_tot_mou_per_call_max,marital_fe_avg_mou_per_call_max,marital_callwait_Mean_max,marital_lor_mean,marital_iwylis_vce_Mean_mean,marital_custcare_Mean_max,marital_income_mean,marital_fe_mean_per_minute_charge_mean,marital_drop_blk_Mean_mean,marital_fe_tot_mean_failed_calls_mean,marital_months_min,marital_fe_mean_extra_amount_they_pay_mean,marital_vceovr_Mean_mean,marital_income_max,marital_ovrrev_Mean_mean,marital_blck_vce_Mean_max,marital_cc_mou_Mean_max,marital_mouiwylisv_Mean_mean,marital_lor_max,marital_ccrndmou_Mean_max,marital_drop_vce_Mean_max,marital_fe_mean_per_minute_charge_max,marital_owylis_vce_Mean_mean,marital_fe_mean_profit_mean,marital_mouowylisv_Mean_mean,marital_totmrc_Mean_min,marital_months_mean,marital_inonemin_Mean_mean,marital_avg3qty_min,marital_unan_vce_Mean_mean,marital_iwylis_vce_Mean_max,marital_change_rev_max,marital_avg6qty_min,marital_fe_tot_mean_failed_calls_max,marital_avg3rev_min,marital_rev_M

In [259]:
telo_df.select_dtypes('object').apply(pd.Series.nunique,axis=0)

new_cell             3
crclscod            54
asl_flag             2
prizm_social_one     6
area                20
dualband             5
refurb_new           3
hnd_webcap           4
ownrent              3
dwlltype             3
marital              6
infobase             3
HHstatin             7
dwllsize            16
ethnic              18
kid0_2               3
kid3_5               3
kid6_10              3
kid11_15             3
kid16_17             3
creditcd             3
dtype: int64

In [260]:
#Creating Dummy Variable for Categorical coumns.
telo_df=dummy(telo_df)
print(telo_df.shape)
gc.collect()
telo_df.head()

(100000, 267)


,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,opk_dat_Mean,mou_opkv_Mean,mou_opkd_Mean,drop_blk_Mean,attempt_Mean,complete_Mean,callfwdv_Mean,callwait_Mean,churn,months,uniqsubs,actvsubs,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,rv,lor,adults,income,numbcars,forgntvl,eqpdays,Customer_ID,fe_mean_per_minute_charge,fe_mean_extra_amount_they_pay,fe_mean_allocated_calls,fe_mean_profit,fe_tot_mean_failed_calls,fe_perc_of_success_calls,fe_mou_cvce_diff_vce_mean,fe_mou_rvce_diff_vce_mean,fe_tot_revenue_per_call,fe_tot_mou_per_call,fe_tot_charge_per_call,fe_tot_revenue_adj,fe_tot_mou_adj,fe_tot_calls_adj,fe_avg_revenue_per_call,fe_avg_mou_per_call,fe_avg_charge_per_call,HHstatin_A,HHstatin_B,HHstatin_C,HHstatin_G,HHstatin_H,HHstatin_I,HHstatin_Missing,area_ATLANTIC_SOUTH_AREA,area_CALIFORNIA_NORTH_AREA,area_CENTRAL_SOUTH_TEXAS_AREA,area_CHICAGO_AREA,area_DALLAS_AREA,area_DC_MARYLAND_VIRGINIA_AREA,area_GREAT_LAKES_AREA,area_HOUSTON_AREA,area_LOS_ANGELES_AREA,area_MIDWEST_AREA,area_Missing,area_NEW_ENGLAND_AREA,area_NEW_YORK_CITY_AREA,area_NORTH_FLORIDA_AREA,area_NORTHWEST_ROCKY_MOUNTAIN_AREA,area_OHIO_AREA,area_PHILADELPHIA_AREA,area_SOUTH_FLORIDA_AREA,area_SOUTHWEST_AREA,area_TENNESSEE_AREA,asl_flag_N,asl_flag_Y,crclscod_A,crclscod_A2,crclscod_A3,crclscod_AA,crclscod_B,crclscod_B2,crclscod_BA,crclscod_C,crclscod_C2,crclscod_C5,crclscod_CA,crclscod_CC,crclscod_CY,crclscod_D,crclscod_D2,crclscod_D4,crclscod_D5,crclscod_DA,crclscod_E,crclscod_E2,crclscod_E4,crclscod_EA,crclscod_EC,crclscod_EF,crclscod_EM,crclscod_G,crclscod_GA,crclscod_GY,crclscod_H,crclscod_I,crclscod_IF,crclscod_J,crclscod_JF,crclscod_K,crclscod_L,crclscod_M,crclscod_O,crclscod_P1,crclscod_S,crclscod_TP,crclscod_U,crclscod_U1,crclscod_V,crclscod_V1,crclscod_W,crclscod_Y,crclscod_Z,crclscod_Z1,crclscod_Z2,crclscod_Z4,crclscod_Z5,crclscod_ZA,crclscod_ZF,crclscod_ZY,creditcd_Missing,creditcd_N,creditcd_Y,dualband_Missing,dualband_N,dualband_T,dualband_U,dualband_Y,dwllsize_A,dwllsize_B,dwllsize_C,dwllsize_D,dwllsize_E,dwllsize_F,dwllsize_G,dwllsize_H,dwllsize_I,dwllsize_J,dwllsize_K,dwllsize_L,dwllsize_M,dwllsize_Missing,dwllsize_N,dwllsize_O,dwlltype_M,dwlltype_Missing,dwlltype_S,ethnic_B,ethnic_C,ethnic_D,ethnic_F,ethnic_G,ethnic_H,ethnic_I,ethnic_J,ethnic_M,ethnic_Missing,ethnic_N,ethnic_O,ethnic_P,ethnic_R,ethnic_S,ethnic_U,ethnic_X,ethnic_Z,hnd_webcap_Missing,hnd_webcap_UNKW,hnd_webcap_WC,hnd_webcap_WCMB,infobase_M,infobase_Missing,infobase_N,kid0_2_Missing,kid0_2_U,kid0_2_Y,kid11_15_Missing,kid11_15_U,kid11_15_Y,kid16_17_Missing,kid16_17_U,kid16_17_Y,kid3_5_Missing,kid3_5_U,kid3_5_Y,kid6_10_Missing,kid6_10_U,kid6_10_Y,marital_A,marital_B,marital_M,marital_Missing,marital_S,marital_U,new_cell_N,new_cell_U,new_cell_Y,ownrent_Missing,ownrent_O,ownrent_R,prizm_social_one_C,prizm_social_one_Missing,prizm_social_one_R,prizm_social_one_S,prizm_social_one_T,prizm_social_one_U,refurb_new_Missing,refurb_new_N,refurb_new_R
0,23.9975,219.25,22.500,0.2475,0.00,0.0,0.0,0.0,0.0,-157.25,-18.9975,0.666667,0.0,0.666667,0.0,6.333333,0.0,52.333333,0.0,42.333333,0.0,45.000000,0.0,0.000000,0.000000,0.000000,18.000000,0.000000,90.643333,0.0,97.176667,0.000000,0.000000,0.000000,0.000000,58.000000,0.0,132.600000,0.0,24.000000,0.0,55.220000,0.0,1.333333,52.333333,45.000000,0.0,0.333333,1,61,2,1,1652,4228.00000,1504.62,1453.44,4085.00,1602,29.66,83.37,32.69,272,116,30,322.0,136.0,38.0,149.989990,2.0,2.0,0.0,0.0,15.0,1.0

In [261]:
# Removing the top 5% and bottom 5% of the outliers
data_outlier(telo_df,0.10,0.90)
print(telo_df.shape)

(100000, 267)


In [262]:
#Removing Zero Varinace columns.
telo_df=rem_var(telo_df,0.01)
print(telo_df.shape)
gc.collect()
telo_df.head()

Your selected dataframe has 265 columns.
Your selected dataframe has 265 Numerical columns.
There are 184 columns has variance greater than 0.01 Percentage
(100000, 186)


,Customer_ID,churn,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,blck_vce_Mean,unan_vce_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,mou_peav_Mean,opk_vce_Mean,mou_opkv_Mean,drop_blk_Mean,attempt_Mean,complete_Mean,callwait_Mean,months,uniqsubs,actvsubs,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,lor,adults,income,numbcars,eqpdays,fe_mean_per_minute_charge,fe_mean_extra_amount_they_pay,fe_mean_allocated_calls,fe_mean_profit,fe_tot_mean_failed_calls,fe_mou_cvce_diff_vce_mean,fe_mou_rvce_diff_vce_mean,fe_tot_revenue_per_call,fe_tot_mou_per_call,fe_tot_charge_per_call,fe_tot_revenue_adj,fe_tot_mou_adj,fe_tot_calls_adj,fe_avg_revenue_per_call,fe_avg_mou_per_call,fe_avg_charge_per_call,HHstatin_A,HHstatin_B,HHstatin_C,HHstatin_G,HHstatin_I,HHstatin_Missing,area_ATLANTIC_SOUTH_AREA,area_CALIFORNIA_NORTH_AREA,area_CENTRAL_SOUTH_TEXAS_AREA,area_CHICAGO_AREA,area_DALLAS_AREA,area_DC_MARYLAND_VIRGINIA_AREA,area_GREAT_LAKES_AREA,area_HOUSTON_AREA,area_LOS_ANGELES_AREA,area_MIDWEST_AREA,area_NEW_ENGLAND_AREA,area_NEW_YORK_CITY_AREA,area_NORTH_FLORIDA_AREA,area_NORTHWEST_ROCKY_MOUNTAIN_AREA,area_OHIO_AREA,area_PHILADELPHIA_AREA,area_SOUTH_FLORIDA_AREA,area_SOUTHWEST_AREA,area_TENNESSEE_AREA,asl_flag_N,asl_flag_Y,crclscod_A,crclscod_AA,crclscod_B,crclscod_BA,crclscod_C,crclscod_CA,crclscod_DA,crclscod_EA,crclscod_ZA,creditcd_Missing,creditcd_N,creditcd_Y,dualband_N,dualband_T,dualband_Y,dwllsize_A,dwllsize_B,dwllsize_C,dwllsize_J,dwllsize_Missing,dwllsize_O,dwlltype_M,dwlltype_Missing,dwlltype_S,ethnic_B,ethnic_F,ethnic_G,ethnic_H,ethnic_I,ethnic_J,ethnic_Missing,ethnic_N,ethnic_O,ethnic_R,ethnic_S,ethnic_U,ethnic_Z,hnd_webcap_Missing,hnd_webcap_WC,hnd_webcap_WCMB,infobase_M,infobase_Missing,kid0_2_Missing,kid0_2_U,kid0_2_Y,kid11_15_Missing,kid11_15_U,kid11_15_Y,kid16_17_Missing,kid16_17_U,kid16_17_Y,kid3_5_Missing,kid3_5_U,kid3_5_Y,kid6_10_Missing,kid6_10_U,kid6_10_Y,marital_A,marital_B,marital_M,marital_Missing,marital_S,marital_U,new_cell_N,new_cell_U,new_cell_Y,ownrent_Missing,ownrent_O,ownrent_R,prizm_social_one_C,prizm_social_one_Missing,prizm_social_one_R,prizm_social_one_S,prizm_social_one_T,prizm_social_one_U,refurb_new_N,refurb_new_R
0,1000001,1,25.1150,219.250,22.500,0.2475,0.00,0.0,0.0,0.0,-157.25,-18.99750,0.666667,0.666667,6.333333,52.333333,0.0,42.333333,45.000000,0.0,0.000000,0.000000,0.000000,18.000000,0.000000,90.643333,97.176667,0.000000,0.000000,0.000000,0.000000,58.000000,132.600000,24.000000,55.220000,1.333333,52.333333,45.000000,0.333333,33.0,2.0,1.0,1652.0,4228.000000,1504.620,1453.440,4085.0,1602.0,29.88,83.370,32.69,272.0,116.0,30.0,322.0,136.0,38.0,149.989990,2.0,2.0,0.0,13.0,1.0,4.0,2.0,361.0,9.136368,1.49750,219.25,0.0,1.333333,45.643333,54.843333,0.910787,2.559322,2.330997,51.18,143.00000,50.0,0.907311,2.550321,2.313935,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0
1,1000002,0,57.4925,482.750,37.425,0.2475,22.75,9.1,9.1,0.0,205.75,22.22275,8.333333,1.000000,61.333333,263.333333,0.0,69.000000,193.333333,0.0,1.666667,6.333333,5.463333,53.000000,0.333333,189.396667,55.280000,46.333333,24.216667,6.333333,3.696667,83.666667,75.333333,157.000000,169.343333,9.333333,263.333333,193.333333,4.666667,33.0,1.0,1.0,6202.0,16500.004667,1914.998,1833.562,16296.1,6124.0,51.53,479.400,265.89,305.0,158.0,40.0,477.0,275.0,48.0,109.757449,3.0,3.0,1.0,1.0,1.0,5.0,1.0,240.0,8.396747,20.06750,460.00,18.2,9.333333,-0.003333,-1.333333,0.207909,1.814550,0.459468,31.92,33.00000,30.0,0.193802,1.815216,0.420394

In [263]:
#Removing Correlated Columns.
telo_df=correlated(telo_df,0.80)
print(telo_df.shape)
gc.collect()
telo_df.head()

Your selected dataframe has 186 columns.
In that dataframe has 184 Numerical columns.
There are 55 columns has correlation greater than 0.8 Percentage
(100000, 129)
(100000, 131)


,Customer_ID,churn,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,blck_vce_Mean,unan_vce_Mean,plcd_dat_Mean,custcare_Mean,threeway_Mean,iwylis_vce_Mean,callwait_Mean,months,uniqsubs,actvsubs,totcalls,totrev,hnd_price,phones,truck,lor,adults,income,numbcars,eqpdays,fe_mean_per_minute_charge,fe_tot_revenue_per_call,fe_tot_mou_per_call,fe_tot_revenue_adj,fe_tot_mou_adj,HHstatin_A,HHstatin_B,HHstatin_C,HHstatin_G,HHstatin_I,HHstatin_Missing,area_ATLANTIC_SOUTH_AREA,area_CALIFORNIA_NORTH_AREA,area_CENTRAL_SOUTH_TEXAS_AREA,area_CHICAGO_AREA,area_DALLAS_AREA,area_DC_MARYLAND_VIRGINIA_AREA,area_GREAT_LAKES_AREA,area_HOUSTON_AREA,area_LOS_ANGELES_AREA,area_MIDWEST_AREA,area_NEW_ENGLAND_AREA,area_NEW_YORK_CITY_AREA,area_NORTH_FLORIDA_AREA,area_NORTHWEST_ROCKY_MOUNTAIN_AREA,area_OHIO_AREA,area_PHILADELPHIA_AREA,area_SOUTH_FLORIDA_AREA,area_SOUTHWEST_AREA,area_TENNESSEE_AREA,asl_flag_N,asl_flag_Y,crclscod_A,crclscod_AA,crclscod_B,crclscod_BA,crclscod_C,crclscod_CA,crclscod_DA,crclscod_EA,crclscod_ZA,creditcd_Missing,creditcd_N,creditcd_Y,dualband_N,dualband_T,dualband_Y,dwllsize_A,dwllsize_B,dwllsize_C,dwllsize_J,dwllsize_Missing,dwllsize_O,dwlltype_M,ethnic_B,ethnic_F,ethnic_G,ethnic_H,ethnic_I,ethnic_J,ethnic_N,ethnic_O,ethnic_R,ethnic_S,ethnic_U,ethnic_Z,hnd_webcap_Missing,hnd_webcap_WC,hnd_webcap_WCMB,infobase_M,infobase_Missing,kid0_2_U,kid0_2_Y,kid11_15_U,kid11_15_Y,kid16_17_U,kid16_17_Y,kid3_5_U,kid3_5_Y,kid6_10_U,kid6_10_Y,marital_A,marital_B,marital_M,marital_S,marital_U,new_cell_N,new_cell_U,new_cell_Y,ownrent_Missing,ownrent_O,ownrent_R,prizm_social_one_C,prizm_social_one_Missing,prizm_social_one_R,prizm_social_one_S,prizm_social_one_T,prizm_social_one_U,refurb_new_N,refurb_new_R
0,1000001,1,25.1150,219.250,22.500,0.2475,0.00,0.0,-157.25,-18.99750,0.666667,0.666667,6.333333,0.0,0.000000,0.000000,0.000000,0.333333,33.0,2.0,1.0,1652.0,1504.620,149.989990,2.0,0.0,13.0,1.0,4.0,2.0,361.0,9.136368,0.910787,2.559322,51.18,143.00000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0
1,1000002,0,57.4925,482.750,37.425,0.2475,22.75,0.0,205.75,22.22275,8.333333,1.000000,61.333333,0.0,1.666667,0.333333,6.333333,4.666667,33.0,1.0,1.0,6202.0,1914.998,109.757449,3.0,1.0,1.0,1.0,5.0,1.0,240.0,8.396747,0.207909,1.814550,31.92,33.00000,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0
2,1000003,1,25.1150,48.325,18.545,0.0000,0.00,0.0,-4.25,0.00000,0.333333,0.000000,2.666667,0.0,0.000000,0.000000,0.000000,0.000000,33.0,1.0,1.0,6202.0,1914.998,29.989990,2.0,0.0,7.0,2.0,5.0,2.0,742.0,1.550517,0.272796,3.085544,113.54,82.00333,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0
3,1000004,0,38.0000,48.325,38.000,0.0000,0.00,0.0,-1.50,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,33.0,1.0,1.0,1502.0,1914.998,29.989990,1.0,0.0,6.0,4.0,6.0,1.0,742.0,1.550517,1.267517,2.040613,59.09,30.00000,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0
4,1000005,0,55.2300,570.500,71.980,0.0000,0.00,0.0,38.50,0.00000,9.666667,0.666667,67.333333,0.0,5.333333,0.000000,0.666667,0.000000,33.0,1.0,1.0,4485.0,1914.998,149.989990,3.0,0.0,5.0,1.0,6.0,1.0,434.0,10.329531,0.486314,3.127759,31.92,63.00000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0


In [264]:
#Writing the data for Modelling
data_path=Path(data_dir,'data/satish/prep_telo_df.csv')
telo_df.to_csv(data_path,index=False)